# Reinforcement Learning
Author: Jin Yeom (jinyeom@utexas.edu)

In this notebook, we'll study and experiment with basics of reinforcement learning (RL).
Note that this notebook is loosely based on Sutton and Barto's book __Reinforcement Learning: An introduction__.

## Q-learning